# Language
This notebook explains the Language part of the tutorial *Easy answer set programming*.

## A note on Easy Answer Set Programming

To make things easy, for most of this tutorial we are only going to write non-recursive programs.

A program is non-recursive if we can write its rules in order, 
and we say that the rules are written in order if 
the predicates that occur in the body of any rule `r` do not occur in the head of any rule that is written after `r`.

The answer sets of a non-recursive program are very easy to define:
they are the result of applying the rules in order.

Following the approach of Easy Answer Set Programming, 
we will always write the rules in order.
In this way, the answer sets of a logic program are simply the result of applying the rules in the order in which they are written.

# The Student Course Timetabling Problem

Imagine that you are a student and you have to choose your courses for the next semester: 
this is the Student Course Timetabling Problem!

We will use this problem to illustrate different aspects of the language of (Easy) Answer Set Programming.

## Facts 

To begin with, we write down some facts representing the information about the available courses:
* Every course has a given number of credits.
* Each lecture of every course takes place at some time period of some day.

We assume that every day is divided into some time periods of a fixed time length.

In [3]:
%%file facts.lp

% credits(C,Cr): the course C has Cr credits
credits( asp,9).
credits(  ml,9).
credits(ling,6).
credits(phil,9).
% ...

% lecture(C,D,P): the course C has a lecture the day D at period P
lecture(asp,thu,3). lecture(asp,fri,3).
lecture(ml,wed,2). lecture(ml,thu,3). lecture(ml,fri,4).
lecture(ling,tue,1). lecture(ling,thu,2). 
lecture(phil,tue,3). lecture(phil,wed,4).
% ...

Overwriting facts.lp


We write the comments `% ...` to indicate that in the real-world we would have more facts like the ones over the comments.

This program has a unique answer set that contains all the atoms occurring in those facts. 
We can compute it with `clingo` as follows. We use option `-V0` to print a short output.

In [4]:
!clingo facts.lp -V0

credits(asp,9) credits(ml,9) credits(ling,6) credits(phil,9) lecture(asp,thu,3) lecture(asp,fri,3) lecture(ml,wed,2) lecture(ml,thu,3) lecture(ml,fri,4) lecture(ling,tue,1) lecture(ling,thu,2) lecture(phil,tue,3) lecture(phil,wed,4)
SATISFIABLE


Before we try to solve the real problem, we add ourselves a possible choice of courses.

In [5]:
%%file enroll.lp

% enroll(C): I enroll in course C
enroll(asp). enroll(ml). enroll(ling). 
% ...

Overwriting enroll.lp


In the cells below we will add further rules to define properties of this enrollment.

In the end, we will replace these facts about `enroll` by some choice rules, to let `clingo` choose the right selection of courses.

## Variables

We define some basic concepts of the problem using rules with variables. They will be useful later when we write further rules.

We begin defining a `course`, a `day`, a `period`, and a time `slot`. 

For example, the first rule says that for every `C`, `D` an `P`, if `lecture(C,D,P)` is in an answer set, then `course(C)` must also be in that answer set. 

In [6]:
%%file basic_1.lp

course(C) :- lecture(C,D,P).
   day(D) :- lecture(C,D,P).
period(P) :- lecture(C,D,P).
slot(D,P) :- lecture(C,D,P).

Overwriting basic_1.lp


We next define the days where there is some lecture according to our enrollment.

The next rule says that for every `C`, `D` and `P`, if `enroll(C)` and `lecture(C,D,P)` are in an answer set, then `some_lecture(D)` must also be in that answer set. 

In [7]:
%%file basic_2.lp 

% I am enrolled to some lecture on day D
some_lecture(D) :- enroll(C), lecture(C,D,P).

Overwriting basic_2.lp


We run `clingo`:

In [8]:
!clingo facts.lp enroll.lp basic_1.lp basic_2.lp -V0

lecture(asp,thu,3) lecture(asp,fri,3) lecture(ml,wed,2) lecture(ml,thu,3) lecture(ml,fri,4) lecture(ling,tue,1) lecture(ling,thu,2) lecture(phil,tue,3) lecture(phil,wed,4) enroll(asp) enroll(ml) enroll(ling) some_lecture(thu) some_lecture(fri) some_lecture(wed) some_lecture(tue) course(asp) course(ml) course(ling) course(phil) day(thu) day(fri) day(wed) day(tue) period(3) period(2) period(4) period(1) slot(thu,3) slot(fri,3) slot(wed,2) slot(fri,4) slot(tue,1) slot(thu,2) slot(tue,3) slot(wed,4) credits(asp,9) credits(ml,9) credits(ling,6) credits(phil,9)
SATISFIABLE


We can tell `clingo` to show only the atoms of predicate `some_lecture` with `1` argument (written `some_lecture/1`)
adding the following show statement:

In [9]:
%%file show_some_lecture.lp
#show some_lecture/1.

Overwriting show_some_lecture.lp


In [10]:
!clingo facts.lp enroll.lp basic_1.lp basic_2.lp show_some_lecture.lp -V0

some_lecture(thu) some_lecture(fri) some_lecture(wed) some_lecture(tue)
SATISFIABLE


### Safety

A rule is safe if every variable occuring in the rule occurs in some *positive literal* in its body.

`clingo` returns an error if it finds an unsafe rule, like in the next example:

In [11]:
%%clingo - 
course(C).
some_lecture(DD) :- enroll(C), lecture(C,D,P).

clingo version 5.5.1
Reading from -
UNKNOWN

Models       : 0+
Calls        : 1
Time         : 0.000s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.000s


-:1:1-11: error: unsafe variables in:
  course(C):-[#inc_base].
-:1:8-9: note: 'C' is unsafe

-:2:1-47: error: unsafe variables in:
  some_lecture(DD):-[#inc_base];lecture(C,D,P);enroll(C).
-:2:14-16: note: 'DD' is unsafe

*** ERROR: (clingo): grounding stopped because of errors


Observe how `clingo` detects that both rules are unsafe, and tells us what specific variables (`C` and `DD`) make the rule unsafe.

Note: A *positive literal* is an atom, a *negative literal* is an atom preceeded by `not`, and
a *literal* is a positive literal or a negative literal.
In the previous rules we have only seen positive literals.

## Arithmetics

In the next example, we want to know how many hours are we supposed to work per week, according to the students' legislation.

In [12]:
%%clingo - -V0

credits_per_semester(30). % there are 30 credits per semester
hours_per_credit(30).     % every credit amounts to 30 hours of work
lecture_weeks(15).        % there are 15 lecture weeks per semester
exam_weeks(6).            % there are 6 weeks for exams per semester

% hours_per_semester(H) : every semester amounts for H hours of work
hours_per_semester(Cr * H) :- credits_per_semester(Cr), hours_per_credit(H).

% hours_per_week(H): every week of the semester we are supposed to work H hours
hours_per_week(H/(L + E)) :- hours_per_semester(H), lecture_weeks(L), exam_weeks(E).

#show hours_per_semester/1. 
#show hours_per_week/1. 

hours_per_semester(900) hours_per_week(42)
SATISFIABLE


We obtain a total of `900` hours per semester and `42` per week. 

The symbol `*` is used for multiplication, `+` is used for addition, and `/` is used for integer division. 
Other arithmetic operators available in `clingo` are
substraction (`-`),
modulo (`\`),
exponentiation (`**`) and
absolute value (`|.|`). You can see them in the next example:

In [13]:
%%clingo - -V0 
left(7).
right(2).
minus ( L -  R ) :- left(L), right(R).
modulo( L \  R ) :- left(L), right(R).
power ( L ** R ) :- left(L), right(R).
abs   (|  -  R|) :-          right(R).

right(2) left(7) minus(5) modulo(1) power(49) abs(2)
SATISFIABLE


## Constants

In the example of `Arithmetics` we have stated that the hours per credit are `30`.
But actually, the hours per credit may range between 25 and 30.

We can then replace `hours_per_credit(30).`
by 
`hours_per_credit(25).` 
and then we obtain `750` hours per semester and `35` per week.

To make these kind of changes easier, we can instead replace 
`hours_per_credit(30).`
by 
`hours_per_credit(hpc).` 
where `hpc` is a constant whose value can be set using `clingo` option `-c hpc=30` or `-c hpc=25`.

In [14]:
%%clingo - -V0 -c hpc=30
 
credits_per_semester(30). % there are 30 credits per semester
hours_per_credit(hpc).    % every credit amounts to hpc hours of work

credits_per_semester(30) hours_per_credit(30)
SATISFIABLE


Additionally, we can write the directive 
`#const hpc=25.` 
in the logic program and then `hpc` is replaced by the integer `25` unless some 
option `-c hpc=...` is used.

In [15]:
%%clingo - -V0
 
credits_per_semester(30). % there are 30 credits per semester
hours_per_credit(hpc).    % every credit amounts to hpc hours of work

#const hpc=25.

credits_per_semester(30) hours_per_credit(25)
SATISFIABLE


## Comparison predicates

We want to identify if we have some conflict between our courses at any time slot. 
We can *try* to do it with the next rule, that says that: 
* For every `C1`, `C2`, `D` and `P`, if `enroll(C1)`, `lecture(C,D,P)`, `enroll(C2)`, and `lecture(C2,D,P)` are in an answer set, then `conflict(D,P)` must also be in that answer set. 

In [16]:
%%file comparison_1.lp

% there is a conflict on day D at period P
conflict(D,P) :- enroll(C1), lecture(C1,D,P), 
                 enroll(C2), lecture(C2,D,P).
#show conflict/2.

Overwriting comparison_1.lp


Recall our facts using clingo:

In [17]:
!clingo -V0 facts.lp enroll.lp

enroll(asp) enroll(ml) enroll(ling) credits(asp,9) credits(ml,9) credits(ling,6) credits(phil,9) lecture(asp,thu,3) lecture(asp,fri,3) lecture(ml,wed,2) lecture(ml,thu,3) lecture(ml,fri,4) lecture(ling,tue,1) lecture(ling,thu,2) lecture(phil,tue,3) lecture(phil,wed,4)
SATISFIABLE


Then `clingo` tell us that...

In [18]:
!clingo facts.lp enroll.lp comparison_1.lp 0 -V0

conflict(thu,3) conflict(fri,3) conflict(wed,2) conflict(fri,4) conflict(tue,1) conflict(thu,2)
SATISFIABLE


... we have conflicts all over the week. But this cannot be correct! For example, on Thursday at the 2nd period we only have the linguistics course, hence no conflict can occur at that time slot. 

The problem is that in the rule above nothing forbids `C1` and `C2` to refer to the same course. Then, we derive a `conflict` for every time slot of every lecture to which we have enrolled. We can fix the problem forcing those two variable to be distinct with the comparison predicate `!=`:

In [19]:
%%file comparison_2.lp 
conflict(D,P) :- enroll(C1), lecture(C1,D,P), 
                 enroll(C2), lecture(C2,D,P), C1 != C2.
#show conflict/2.

Overwriting comparison_2.lp


In [20]:
!clingo facts.lp enroll.lp comparison_2.lp 0 -V0

conflict(thu,3)
SATISFIABLE


Now the result is correct: we have a conflict on Thursday at the 3rd period.

You can replace the symbol `!=` by `<` or by `>` and see that the result is the same. This shows that in `clingo` the constants (like `mon` or `thu`) are ordered. 
In fact, they are ordered lexicographically, but this order is not always meaningful to us. 
To see this, let's *try* to define when day `D1` is `before` day `D2`:

In [21]:
%%clingo - -V0

day(mon). day(tue). day(wed). day(thu). day(fri).
before(D1,D2) :- day(D1), day(D2), D1 < D2.

#show before/2.

before(fri,mon) before(mon,tue) before(thu,tue) before(fri,tue) before(mon,wed) before(tue,wed) before(thu,wed) before(fri,wed) before(mon,thu) before(fri,thu)
SATISFIABLE


This doesn't work nicely. For example, `fri`day should not be before `mon`day. 
In this case, it is better to use numbers to refer directly to the elements we want to order:

In [22]:
%%clingo - -V0

day(1). day(2). day(3). day(4). day(5).
before(D1,D2) :- day(D1), day(D2), D1 < D2.

#show before/2.

before(1,2) before(1,3) before(2,3) before(1,4) before(2,4) before(3,4) before(1,5) before(2,5) before(3,5) before(4,5)
SATISFIABLE


or use some additional predicate to define that order, like `position/2` in the next example:

In [23]:
%%clingo - -V0

day(mon). day(tue). day(wed). day(thu). day(fri).
position(mon,1). position(tue,2). position(wed,3). position(thu,4). position(fri,5).
before(D1,D2) :- day(D1), day(D2), position(D1,P1), position(D2,P2), P1 < P2.

#show before/2.

before(mon,tue) before(mon,wed) before(tue,wed) before(mon,thu) before(tue,thu) before(wed,thu) before(mon,fri) before(tue,fri) before(wed,fri) before(thu,fri)
SATISFIABLE


Let's move one and see more comparison operators at work. 
We want to define the time slots that are early, at the first period of the day, given our enrollment.
We can do this with any of the rules of the following program.
You can uncomment each of them individually and see that they lead to the same result:

In [24]:
%%clingo - facts.lp enroll.lp -V0

% uncomment one
  early(C,D) :- enroll(C), lecture(C,D,1).
% early(C,D) :- enroll(C), lecture(C,D,P), P  = 1.
% early(C,D) :- enroll(C), lecture(C,D,P), P <= 1.
% #const early_time=2.
% early(C,D) :- enroll(C), lecture(C,D,P), P  < early_time. % uncomment also the line above
% early(C,D) :- enroll(C), lecture(C,D,P), P != 2, P != 3, P != 4.

#show early/2.

early(ling,tue)
SATISFIABLE


## Intervals

Above we have defined the days of the week using numbers:

In [25]:
%%clingo - -V0
day(1). day(2). day(3). day(4). day(5).

day(1) day(2) day(3) day(4) day(5)
SATISFIABLE


We can do the same more compactly, using intervals in the head:

In [26]:
%%clingo - -V0
day(1..5).

day(1) day(2) day(3) day(4) day(5)
SATISFIABLE


or in the body:

In [27]:
%%clingo - -V0
day(D) :- D=1..5.

day(1) day(2) day(3) day(4) day(5)
SATISFIABLE


The bounds of the interval can be variables, like here:

In [28]:
%%clingo - -V0
min(1). max(5).
day(Min..Max) :- min(Min), max(Max).

max(5) min(1) day(1) day(2) day(3) day(4) day(5)
SATISFIABLE


or here:

In [29]:
%%clingo - -V0
min(1). max(5).
day(D) :- D=Min..Max, min(Min), max(Max).

max(5) min(1) day(1) day(2) day(3) day(4) day(5)
SATISFIABLE


More than one interval can occur in one rule, 
like in the next rules defining time slots:

In [30]:
%%clingo - -V0
slot(1..5,1..4).

slot(1,1) slot(2,1) slot(3,1) slot(4,1) slot(5,1) slot(1,2) slot(2,2) slot(3,2) slot(4,2) slot(5,2) slot(1,3) slot(2,3) slot(3,3) slot(4,3) slot(5,3) slot(1,4) slot(2,4) slot(3,4) slot(4,4) slot(5,4)
SATISFIABLE


In [31]:
%%clingo - -V0
slot(D,P) :- D=1..5, P=1..4.

slot(1,1) slot(2,1) slot(3,1) slot(4,1) slot(5,1) slot(1,2) slot(2,2) slot(3,2) slot(4,2) slot(5,2) slot(1,3) slot(2,3) slot(3,3) slot(4,3) slot(5,3) slot(1,4) slot(2,4) slot(3,4) slot(4,4) slot(5,4)
SATISFIABLE


## Pooling

Another useful feature of our language is pooling. Remember our definition of the days of the week:

In [32]:
%%clingo - -V0
day(mon). day(tue). day(wed). day(thu). day(fri).

day(mon) day(tue) day(wed) day(thu) day(fri)
SATISFIABLE


We can write it simply like this:

In [33]:
%%clingo - -V0
day(mon;tue;wed;thu;fri).

day(mon) day(tue) day(wed) day(thu) day(fri)
SATISFIABLE


We can also use pools to define the time slots with day names:

In [34]:
%%clingo - -V0 0

period(1..4).
slot(mon,P;tue,P;wed,P;thu,P;fri,P) :- period(P).

#show slot/2.

slot(fri,1) slot(fri,2) slot(fri,3) slot(fri,4) slot(thu,1) slot(thu,2) slot(thu,3) slot(thu,4) slot(wed,1) slot(wed,2) slot(wed,3) slot(wed,4) slot(tue,1) slot(tue,2) slot(tue,3) slot(tue,4) slot(mon,1) slot(mon,2) slot(mon,3) slot(mon,4)
SATISFIABLE


## Negation

Let's define which courses are nice for us because they do not have any early lecture at the first period of some day.
This is our first attempt:

In [35]:
%%clingo - -V0 facts.lp

% course C is nice on day D
nice(C,D) :- lecture(C,D,P), P = 2..4.

% course C is nice
nice(C) :- nice(C,mon), nice(C,tue), nice(C,wed), nice(C,thu), nice(C,fri).

#show nice/1.


SATISFIABLE


The result tells us that there is no nice course! But this cannot be true. Let's see again the lectures...

In [36]:
%%clingo - -V0 facts.lp
#show lecture/3.

lecture(asp,thu,3) lecture(asp,fri,3) lecture(ml,wed,2) lecture(ml,thu,3) lecture(ml,fri,4) lecture(ling,tue,1) lecture(ling,thu,2) lecture(phil,tue,3) lecture(phil,wed,4)
SATISFIABLE


The only course that is not nice, according to our informal definition, is `ling`, because we have `lecture(ling,tue,1)`. 

The problem with our rules is that for a course `C` to be nice we require that *every day* there is some nice lecture.

Actually, we would like to say that `C` is nice if there is no day that is early. We can do this using negation:

In [37]:
%%clingo - -V0 facts.lp basic_1.lp

% course C is early on day D
early(C,D) :- lecture(C,D,1).

% course C is nice
nice(C) :- course(C), 
           not early(C,mon), not early(C,tue), not early(C,wed), not early(C,thu), not early(C,fri).

#show nice/1.

nice(asp) nice(ml) nice(phil)
SATISFIABLE


Now we obtain the right answer!

However, the rule defining `nice(C)` looks too complicated, and it does not work if there are lectures on Saturdays... 

Let's give it another try:

In [38]:
%%clingo - -V0 facts.lp basic_1.lp

% course C is early (some day)
early(C) :- lecture(C,D,1).

% course C is nice
nice(C) :- course(C), not early(C).

#show nice/1.

nice(asp) nice(ml) nice(phil)
SATISFIABLE


This looks much simpler, and it is closer to our description in natural language of a nice course.

### Safety

Now you could be asking yourself: why do we need to add the atom `course(C)` to the body of the rules for `nice(C)`? Let's see what `clingo` says if we delete it from the rules:

In [39]:
%%clingo - -V0 facts.lp basic_1.lp

% course C is early on day D
early(C,D) :- lecture(C,D,1).

% course C is nice
nice(C) :- 
           not early(C,mon), not early(C,tue), not early(C,wed), not early(C,thu), not early(C,fri).
    
% course C is early (some day)
early(C) :- lecture(C,D,1).

% course C is nice
nice(C) :- not early(C).

UNKNOWN


-:6:1-7:101: error: unsafe variables in:
  nice(C):-[#inc_base];not early(C,fri);not early(C,thu);not early(C,wed);not early(C,tue);not early(C,mon).
-:6:6-7: note: 'C' is unsafe

-:13:1-25: error: unsafe variables in:
  nice(C):-[#inc_base];not early(C).
-:13:6-7: note: 'C' is unsafe

*** ERROR: (clingo): grounding stopped because of errors


`clingo` gives an error and tells us that now the rules are not safe!

Remember that a rule is safe if every variable occurring in the rule occurs in some *positive literal* in its body. 

Without the atom `course(C)`, the variable `C` does not occur in any positive literal of the bodies, and the rules are unsafe.

## Anonymous variables

We have defined a time slot with a rule like this:

In [40]:
%%clingo - -V0 facts.lp

slot(D,P) :- lecture(C,D,P).

#show slot/2.

slot(thu,3) slot(fri,3) slot(wed,2) slot(fri,4) slot(tue,1) slot(thu,2) slot(tue,3) slot(wed,4)
SATISFIABLE


The variable `C` occurs only in the body of the rule, in the atom `lecture(C,D,P)`. It is not used anywhere else. In this case, we can replace the variable `C` by an anonymous variable `_`.

In [41]:
%%clingo - -V0 facts.lp

slot(D,P) :- lecture(_,D,P).

#show slot/2.

slot(thu,3) slot(fri,3) slot(wed,2) slot(fri,4) slot(tue,1) slot(thu,2) slot(tue,3) slot(wed,4)
SATISFIABLE


In the other direction, we can always replace an anonymous variable by a new variable that does not occur anywhere in the rule. 
For example, in the last program we can replace `_` by `X`, or by `C` again.

When we see an anonymous variable, we know that the value of the argument at the position of the variable is not relevant.

Using anonymous variables or not is mainly a matter of style. 

### Implementation

`clingo` handles anonymous variables by adding an auxiliary atom. 
For example, the program above is rewritten by `clingo` into this one:

In [82]:
%%clingo - -V0 facts.lp

% auxiliary atom
aux_lecture(D,P) :- lecture(X,D,P).

% replace lecture(_,D,P) by auxiliary atom aux_lecture(D,P)
slot(D,P) :- aux_lecture(D,P).

#show slot/2.

slot(thu,3) slot(fri,3) slot(wed,2) slot(fri,4) slot(tue,1) slot(thu,2) slot(tue,3) slot(wed,4)
SATISFIABLE


Note that this affects the meaning of anonymous variables in negative literals. To see this, let's define a nice enrollment with these rules:

In [83]:
%%clingo - -V0 facts.lp basic_1.lp

% course C is early (some day)
early(C) :- lecture(C,D,1).

% the enrollment is nice if there is no early course
nice :- not early(_).

#show nice/0.


SATISFIABLE


This looks good. But what happens if we replace the atom `early(_)` by `early(C)`? Try it and you will see that `clingo` gives an error, because now the rule becomes unsafe. But remember that `clingo` is actually replacing the `early(_)` atom by an auxiliary predicate, like here: 

In [85]:
%%clingo - -V0 facts.lp basic_1.lp

% course C is early (some day)
early(C) :- lecture(C,D,1).

% auxiliary atom
aux_early :- early(C).

% replace early(_) by auxiliary atom aux_early
nice :- not aux_early.

#show nice/0.


SATISFIABLE


Observe that the program is safe, and we obtain the same solution as before.

## Cardinality constraints

In the next step, we want to know if we have enrolled in many courses. And let's say that by many courses we mean at least `2` courses. We can write this rule for that:

In [105]:
%%clingo - -V0 enroll.lp

% enrolled in at least two courses
many_courses :- enroll(C1), enroll(C2), C1 != C2.

enroll(asp) enroll(ml) enroll(ling) many_courses
SATISFIABLE


Good, now we change our mind and say that by many courses we mean at least `3` courses. Let's extend the previous rule for that:

In [106]:
%%clingo - -V0 enroll.lp

% enrolled in at least two courses
many_courses :- enroll(C1), enroll(C2), C1 != C2, 
                enroll(C3), C1 != C3, C2 != C3.

enroll(asp) enroll(ml) enroll(ling) many_courses
SATISFIABLE


This works, but it is getting very tedious... 

Moreover, the approach is not very general. We would like to say something like this: by many courses we mean at least `m` courses, where 
`m` is a constant that we fix from the command line. How could we handle this case?

In [ ]:
%%clingo - -V0 enroll.lp -c m=2

% enrolled in at least m courses
many_courses :- enroll(C1), enroll(C2), C1 != C2, ...

I don't know how to do it! 

Let's try a different approach. 
We would like to talk about how many `enroll(C)` atoms do we have.
We can do exactly this with cardinality constraints.
They allow us to consider the set of all `enroll(C)` atoms in an answer set, 
and specify a constraint on its cardinality.
In our case, that it is at least `m`.

In [108]:
%%clingo - -V0 enroll.lp -c m=2

% enrolled in at least m courses
many_courses :- m { enroll(C) }.

enroll(asp) enroll(ml) enroll(ling) many_courses
SATISFIABLE


The rule says: 
* Consider the set of literals of the form `enroll(C)`  such that `enroll(C)` belongs to an answer set. 
* If the cardinality of that set is greater or equal than `m`, then `many_courses` must be in that answer set.

In this case, the set mentioned in the first bullet is `{ enroll(asp), enroll(ml), enroll(ling) `}. 
Its cardinality is greater than `m`, that is equal to `2` given the option `-c m=2`, and therefore `many_courses` is added to the answer set.
You can change the value of the constant `m` to `4` and see how in that case the atom `many_courses` is not added.

We could also specify the value of `m` using a predicate `many/1`, and we obtain the same result:

In [110]:
%%clingo - -V0 enroll.lp -c m=2

% by many we mean at least m
many(m).

% we are enrolled to many courses
many_courses :- many(M), M { enroll(C) }.

many(2) enroll(asp) enroll(ml) enroll(ling) many_courses
SATISFIABLE


We can read the new rule as follows:
* For every `M`, if `many(M)` is in an answer set:
  * Consider the set of literals of the form `enroll(C)` such that `enroll(C)` belongs to that answer set. 
  * If the cardinality of that set is greater or equal than `M`, then `many_courses` must be in that answer set.

Next, we want to know what days do we have many different courses. Following the previous examples, we could use these rules:

In [112]:
%%clingo - -V0 facts.lp enroll.lp basic.lp -c m=2

% by many we mean at least m
many(m).

% on day D we are enrolled to some lecture(s) of course C
enroll(C,D) :- enroll(C), lecture(C,D,P).

% on day D we are enrolled to lectures of many courses 
many_courses(D) :- day(D), many(M), M { enroll(C,D) }.

#show many_courses/1.

many_courses(thu) many_courses(fri)
SATISFIABLE


That's nice! 

We can read the last rule as follows:
* For every `D` and `M`, if `day(D)` and `many(M)` are in an answer set:
  * Consider the set of literals of the form `enroll(C,D)` such that `enroll(C,D)` belongs to that answer set.
  * If the cardinality of that set is greater or equal than `M`, then `many_courses(D)` must be in that answer set.

We can also represent the last two rules with a single one, like here:

In [114]:
%%clingo - -V0 facts.lp enroll.lp basic.lp -c m=2

% by many we mean at least m
many(m).

% on day D we are enrolled to lectures of many courses 
many_courses(D) :- day(D), many(M), M { enroll(C) : lecture(C,D,P) }.

#show many_courses/1.

many_courses(thu) many_courses(fri)
SATISFIABLE


Nice, we obtain the same result! 

But how do we read the rule now? Only the reading of the cardinality constraint changes:
* For every `D` and `M`, if `day(D)` and `many(M)` are in an answer set:
  * Consider the set of literals of the form `enroll(C)` such that ...
  * ... for some `P`, `enroll(C)` and `lecture(C,D,P)` belong to that answer set.
  * If the cardinality of that set is greater or equal than `M`, then `many_courses(D)` must be in that answer set.
  
Observe that, to add `enroll(C)` to the set to consider,
it is enough to have *one* `P` such that `enroll(C)` and `lecture(C,D,P)` belong to the answer set.
  
To understand this a bit better, let's remember the facts about `enroll/1` and `lecture/3`.

In [116]:
%%clingo - -V0 facts.lp enroll.lp
#show enroll/1.
#show lecture/3.

enroll(asp) enroll(ml) enroll(ling) lecture(asp,thu,3) lecture(asp,fri,3) lecture(ml,wed,2) lecture(ml,thu,3) lecture(ml,fri,4) lecture(ling,tue,1) lecture(ling,thu,2) lecture(phil,tue,3) lecture(phil,wed,4)
SATISFIABLE


When day `D` is `wed` and `M` is `2`, the set that we have to consider is `{ enroll(ml) }`, of cardinality smaller than `M`. Moreover, for other values of `M`, there is no atom `many_courses(M)` in the answer set. This explains why `many_courses(wed)` is not in the answer set.

When day `D` is `thu` and `M` is `2`, the set that we have to consider is `{ enroll(asp), enroll(ml)}`, of cardinality greater or equal than `M`. For this reason, `many_courses(thu)` is added to the answer set.

### Some notes

In the cardinality constraint `M { enroll(C) : lecture(C,D,P) }`, the part `enroll(C) : lecture(C,D,P)` is called *conditional literal*. 
It is  composed of the *(positive) literal* `enroll(C)` and the *condition* `lecture(C,D,P)`. 


**Note** that the literal may be negative. To see this, just replace in the previous program the last rule by this one:
* `many_courses(D) :- day(D), many(M), M { not enroll(C) : lecture(C,D,P) }.`
  
In this case, we consider the set of literals of the form `not enroll(C)` such that, for some `P`, `enroll(C)` does not belong to that answer set and `lecture(C,D,P)` belongs to that answer set. When `D` is `wed` and `M` is `2`, that set is `{ not enroll(phil) }`, and when `D` is `thu` and `M` is `2`, that set is `{}`.

**Note** also that the condition of the conditional literal may have more literals. To see this, just replace in the previous program the last rule by this one:
* `many_courses(D) :- day(D), many(MC), M { enroll(C) : lecture(C,D,P), P >= 3 }.`

In this case, we consider the set of literals of the form `enroll(C)` such that, for some `P`, `enroll(C)` and `lecture(C,D,P)` belong to that answer set and `P >= 3`. When `D` is `wed` and `M` is `2`, that set is `{}`, and when `D` is `thu` and `M` is `2`, that set is `{ enroll(asp), enroll(ml) }`.

**Note** also that the constraint may have more conditional literals. To see this, just replace in the previous program the last rule by this one:
* `many_courses(D) :- day(D), many(M), M { enroll(C) : lecture(C,D,P), P <= 1; enroll(C) : lecture(C,D,P), P >= 4 }.`

In this case, we consider the set of literals that are
*either* of the form `enroll(C)` and, for some `P`, `enroll(C)` and `lecture(C,D,P)` belong to that answer set and `P <= 1`;
*or* of the form `enroll(C)` and, for some `P`, `enroll(C)` and `lecture(C,D,P)` belongs to that answer set and `P >= 4`.
When `D` is `wed` and `M` is `2`, that set is `{}`, and when `D` is `thu` and `M` is `2`, that set is also `{}`.

### Comparison operators

We can state different constraints over sets using the comparison operators `<`, `<=`, `>=` and `>`. 
We can also use the comparison operator `=`, but we will talk about it later.
Observe that having no comparison operator, as in the previous examples, is the same as having the operator `<=`.
Let's see an example with two cardinality constraints represented in different but equivalent ways:

In [144]:
%%clingo - -V0 facts.lp enroll.lp basic.lp -c m=2

% by many we mean m
many(m).

% uncommenting any of the rules below we obtain the same result
% all cardinality constraints express the same constraint over the same set
% on day D we are enrolled to lectures of many courses 
many_courses(D) :- day(D), many(M), M { enroll(C) : lecture(C,D,P) }.
% many_courses(D) :- day(D), many(M), M <= { enroll(C) : lecture(C,D,P) }.
% many_courses(D) :- day(D), many(M), { enroll(C) : lecture(C,D,P) } >= M.
% many_courses(D) :- day(D), many(M), { enroll(C) : lecture(C,D,P) } >  M.

% uncommenting any of the rules below we obtain the same result
% all cardinality constraints express the same constraint over the same set
% on day D we are enrolled to lectures of between 1 and 2 courses
some_courses(D) :- day(D), 1 { enroll(C) : lecture(C,D,P) } 2.
% some_courses(D) :- day(D), 1 <= { enroll(C) : lecture(C,D,P) } 2.
% some_courses(D) :- day(D), 1 <= { enroll(C) : lecture(C,D,P) } <= 2.
% some_courses(D) :- day(D), 1 <= { enroll(C) : lecture(C,D,P) } <  3.
% some_courses(D) :- day(D), 3 >  { enroll(C) : lecture(C,D,P) } >  0.

#show many_courses/1.
#show some_courses/1.

many_courses(thu) many_courses(fri) some_courses(fri) some_courses(wed) some_courses(tue)
SATISFIABLE


We can read the first rule for `some_courses(D)` as follows:
* For every `D`, if `day(D)` is in an answer set:
  * Consider the set of literals of the form `enroll(C)` such that ...
  * ... for some `P`, `enroll(C)` and `lecture(C,D,P)` belong to that answer set.
  * If the cardinality of that set is between `1` and `2`, then `some_courses(D)` must be in that answer set.

### Swapping the literal and the condition

The previous program was telling us what days do we have many different courses.
In this way, if we had two lectures of the same course on the same day, we were counting them only once.
To see this, you can add the fact `lecture(ml,wed,1).` to the previous program and see that we still obtain that 
on `wed`nesdays there are not many courses.

Now we would like to know what days do we have many different *lectures*, 
so that if we have two lectures of the same course on the same day, 
they are counted twice.
We can do this with this program, simply swapping the elements of the conditional literal:

In [117]:
%%clingo - -V0 facts.lp enroll.lp basic.lp -c m=2

% by many we mean at least m
many(m).

% on day D we are enrolled to many lectures of many courses 
many_lectures(D) :- day(D), many(M), M { lecture(C,D,P) : enroll(C) }.

% we add the lecture of ml on wednesdays
lecture(ml,wed,1).

#show many_lectures/1.

many_lectures(thu) many_lectures(fri) many_lectures(wed)
SATISFIABLE


The rule with the cardinality constraint says: 
* For every `D` and `M`, if `day(D)` and `many(M)` are in an answer set:
  * Consider the set of literals of the form `lecture(C,D,P)` such that ...
  * ... for some `P`, `lecture(C,D,P)` and `enroll(C)` belong to that answer set.
  * If the cardinality of that set is greater or equal than `M`, then `many_lectures(D)` must be in that answer set.
  
The only change is that now we do not count the number of literals of the form `enroll(C)`, but the number of literals of the form `lecture(C,D,P)`. Then, for example, when day `D` is `wed` and `M` is `2`, the set to consider is `{ lecture(ml,wed,1), lecture(ml,wed,2) }`, of cardinality greater or equal than `M`, and `many_lectures(wed)` is added to the answer set.

### Local and global variables

In the previous examples, 
the variables `C` and `P` are local to the corresponding conditional literals, 
because they *only* occur inside the cardinality constraint. 
On the other hand, the variables `D` and `M` are global, because they do not occur *only* inside the cardinality constraint.

Inside a conditional literal, we can freely rename the local variables, but not the global ones. 
For example, in the next program, there is no difference between the first rule for `many_lectures` and the second, while the third is different. 
See the different results commenting and uncommenting the rules:

In [118]:
%%clingo - -V0 -c m=2

% facts
day(mon;tue;wed;thu;fri). 
many(m). 
enroll(asp). lecture(asp,mon,1).
enroll(ml).  lecture(ml,mon,1). lecture(ml,mon,4).

% C and P are local to the first conditional literal, and to the second as well
extreme(D) :- day(D), many(M), M { enroll(C) : lecture(C,D,P), P <= 1 ; 
                                   enroll(C) : lecture(C,D,P), P >= 4 }.

% CC and PP are local to the first conditional literal; CC and PP are local to the second
% extreme(D) :- day(D), many(M), M { enroll(CC) : lecture(CC,D, PP),  PP <= 1 ; 
%                                    enroll(CC) : lecture(CC,D,PPP), PPP >= 4 }.

% C, DD and P are local to the first conditional literal; C and P are also local to the second
% extreme(D) :- day(D), many(M), M { enroll(C) : lecture(C,DD,P), P <= 1 ;  
%                                    enroll(C) : lecture(C, D,P), P >= 4 }.

#show extreme/1.

extreme(mon)
SATISFIABLE


The second rule modifies the first rule by renaming the *local* variables `C` and `P` of the conditional literals.
Given that those variables are local, the answer sets using the second rule remain the same as using the first one.
Note also how, in the second rule, 
the `CC` of the first conditional literal is independent of the `CC` of the second one, 
just as `PP` is independent of `PPP`.

The third rule modifies the first rule by renaming the *global* variable `D` *only* inside the first conditional literal.
Given that `D` is global, the answer sets using the third rule may change, and indeed they change in the previous program.
On the other hand, if we also replace in the third rule the other three occurrences of `D` by `DD`, 
then we have the same answer sets as with the first rule.

### Safety

We have to update our definition of safety to handle global and local variables.
We say that a rule with cardinality constraints is safe if:
* every global variable occurring in the rule occurs in some *positive literal* in its body, and
* every variable local to some conditional literal occurs in some *positive literal* of that conditional literal.

You can see how this works looking at the following program and the errors reported by `clingo` below:

In [119]:
%%clingo - -V0

% safe
extreme(D) :- day(D), many(M), M { enroll(C) : lecture(C,D,P), P <= 1 ; 
                                   enroll(C) : lecture(C,D,P), P >= 4 }.

% safe:
extreme(D) :- day(D), many(M), M { enroll(C) : lecture(C,D,P), P <= 1 ; 
                                   not enroll(C) : lecture(C,D,P), P >= 4 }.

% unsafe: D is global and 
%         does not occur in any positive literal in the body
extreme(D) :- not day(D), many(M), M { enroll(C) : lecture(C,D,P), P <= 1 ; 
                                       enroll(C) : lecture(C,D,P), P >= 4 }.

% unsafe: P is local to the second conditional literal and
%         does not occur in a positive literal of that conditional literal
extreme(D) :- day(D), many(M), M { enroll(C) : lecture(C,D,P), P <= 1 ; 
                                   enroll(C) : not lecture(C,D,P), P >= 4 }.

UNKNOWN


-:12:1-13:77: error: unsafe variables in:
  extreme(D):-[#inc_base];many(M);M<=#count{0,enroll(C):lecture(C,D,P),P<=1,enroll(C);0,enroll(C):lecture(C,D,P),P>=4,enroll(C)};not day(D).
-:12:9-10: note: 'D' is unsafe

-:17:32-18:76: error: unsafe variables in:
  M<=#count{0,enroll(C):lecture(C,D,P),P<=1,enroll(C);0,enroll(C):not lecture(C,D,P),P>=4,enroll(C)}
-:18:64-65: note: 'P' is unsafe

*** ERROR: (clingo): grounding stopped because of errors


## Count Aggregates

Count aggregates are similar to cardinality constraints,
but consider sets of tuples of terms instead of sets of literals.

A tuple of terms is just a list of terms separated by commas.
So far, we have seen the following types of terms: integers (`1`), constants (`mon`), and variables (`X`).
Some tuples of terms are `1,mon,X` or `2`.

For example, the cardinality constraint:
* `M { enroll(C) : lecture(C,D,P) }`

considers 
* the set of *literals* of the form `enroll(C) ` such that `enroll(C)` and `lecture(C,D,P)` belong to a given answer answer set,

and expresses the constraint that the cardinality of that set must be at least `M`. 
The cardinality constraint can be replaced by the following count aggregate:
* `M #count { C : enroll(C), lecture(C,D,P) }`

that considers: 
* the set of *tuples* of the form `C` such that `enroll(C)` and `lecture(C,D,P)` belong to a given answer set.

and also expresses the constraint that the cardinality of that set must be at least `M`. 

The part `C : enroll(C), lecture(C,D,P)` is called *aggregate element*. 
It is composed of the tuple of terms `C` and the condition `enroll(C), lecture(C,D,P)`.
An aggregate may have one or more aggregate elements.

Let's see this in action:

In [125]:
%%clingo - -V0 facts.lp enroll.lp basic.lp -c m=2

% by many we mean at least m
many(m).

% uncommenting any of the rules below we obtain the same result
% on day D we are enrolled to lectures of many courses 
many_courses(D) :- day(D), many(M), M { enroll(C) : lecture(C,D,P) }.
% many_courses(D) :- day(D), many(M), M #count { C : enroll(C), lecture(C,D,P) }. 

#show many_courses/1.

many_courses(thu) many_courses(fri)
SATISFIABLE


When day `D` is `wed` and `M` is `2`, 
the set of literals considered by the cardinality constraint is `{ enroll(ml) }`, 
while the set of tuples considered by the count aggregate is `{ ml }`.
The constraints expressed over those sets do not hold because their cardinality is smaller than `MC`.

When day `D` is `thu` and `M` is `2`, the set of literals considered by the cardinality constraint is `{ enroll(asp), enroll(ml)}`, 
and the set of tuples considered by the count aggregate is `{ asp, ml `}.
The constraints expressed over those sets do hold because their cardinality is at least `M`.


### Example: lectures per day

Let's see the example about the number of lectures per day:

In [129]:
%%clingo - -V0 facts.lp enroll.lp basic.lp -c m=2

% by many we mean at least m
many(m).

% uncommenting any of the rules below we obtain the same result
% on day D we are enrolled to many lectures of many courses 
many_lectures(D) :- day(D), many(M), M { lecture(C,D,P) : enroll(C) }.
% many_lectures(D) :- day(D), many(M), M #count { C,D,P : enroll(C), lecture(C,D,P) }.
% many_lectures(D) :- day(D), many(M), M #count { C,P : enroll(C), lecture(C,D,P) }.

% we add the lecture of ml on wednesdays
lecture(ml,wed,1).

#show many_lectures/1.

many_lectures(thu) many_lectures(fri) many_lectures(wed)
SATISFIABLE


When day `D` is `wed` and `M` is `2`, the set of literals considered by the cardinality constraint is `{ lecture(ml,wed,1), lecture(ml,wed,2) }`, 
while the set of tuples considered by the first aggregate is `{ ml,wed,1; ml,wed,2 }` (here we use `;` to separate the tuples) and 
the set of tuples considered by the second aggregate is `{ ml,1; ml,2 }`.
The constraints expressed over those sets hold because their cardinality is at least `M`.

Given that `D` is a global variable, we can replace the tuple `C,D,P` in the first aggregate 
by `C,P` in the second aggregate, and the constraint expressed by the aggregate does not change.
In fact, it does not make sense to use global variables in the tuples of aggregate elements, 
and `clingo` informs us about this usage in the first aggregate.

Observe that we can express different constraints using different tuples and the same condition.
For example, assuming that `D` and `M` are global variables:
* `M #count { C : enroll(C), lecture(C,D,P) }` expresses a constraint over the *courses* `C` to which we have enrolled in day `D`;
* `M #count { C,P : enroll(C), lecture(C,D,P) }` expresses a constraint over the pairs `C,P` of courses `C` and periods `P` 
   (that define a *lecture*) to which we have enrolled in day `D`; and
* `M #count { P : enroll(C), lecture(C,D,P) }` expresses a constraint over the *periods* `P` when there is some course 
  to which we have enrolled in day `D`. 
  
  

### Example: conflicting lectures

Let's see another example, this time about conflicting lectures:

In [131]:
%%clingo - -V0 facts.lp enroll.lp basic.lp -c m=2

% by many we mean m
many(m).

% there is a conflict on day D at period P
conflict(D,P) :- enroll(C1), lecture(C1,D,P), 
                 enroll(C2), lecture(C2,D,P), C1 < C2.
    
% there are conflicts on many courses, days, slots or lectures 
many_conflicts( courses) :- many(M), M #count {     C : conflict(D,P), lecture(C,D,P) }.
many_conflicts(    days) :- many(M), M #count {     D : conflict(D,P), lecture(C,D,P) }.
many_conflicts(   slots) :- many(M), M #count {   D,P : conflict(D,P), lecture(C,D,P) }.
many_conflicts(lectures) :- many(M), M #count { C,D,P : conflict(D,P), lecture(C,D,P) }.

#show many_conflicts/1.

many_conflicts(lectures) many_conflicts(courses)
SATISFIABLE


### Local variables, global variables and safety 

Local variables, global variables and safety are defined similarly as for cardinality constraints. 
Next, you can see the previous example of safety, now  with count aggregates:

In [133]:
%%clingo - -V0

% safe
extreme(D) :- day(D), many(M), M #count { C : enroll(C), lecture(C,D,P), P <= 1 ; 
                                          C : enroll(C), lecture(C,D,P), P >= 4 }.

% safe:
extreme(D) :- day(D), many(M), M #count { C : enroll(C), lecture(C,D,P), P <= 1 ; 
                                          C : not enroll(C), lecture(C,D,P), P >= 4 }.

% unsafe: D is global and 
%         does not occur in any positive literal in the body
extreme(D) :- not day(D), many(M), M #count { C : enroll(C), lecture(C,D,P), P <= 1 ; 
                                              C : enroll(C), lecture(C,D,P), P >= 4 }.

% unsafe: P is local to the second conditional literal and
%         does not occur in a positive literal of that conditional literal
extreme(D) :- day(D), many(M), M #count { C : enroll(C), lecture(C,D,P), P <= 1 ; 
                                          C : enroll(C), not lecture(C,D,P), P >= 4 }.

UNKNOWN


-:12:1-13:87: error: unsafe variables in:
  extreme(D):-[#inc_base];many(M);M<=#count{C:enroll(C),lecture(C,D,P),P<=1;C:enroll(C),lecture(C,D,P),P>=4};not day(D).
-:12:9-10: note: 'D' is unsafe

-:17:32-18:86: error: unsafe variables in:
  M<=#count{C:enroll(C),lecture(C,D,P),P<=1;C:enroll(C),not lecture(C,D,P),P>=4}
-:18:74-75: note: 'P' is unsafe

*** ERROR: (clingo): grounding stopped because of errors


## Sum Aggregates, Conditional Literals, and Optimization Statements

We plan to add sections for them in the future.